In [ ]:
import abc
import numpy as np
import operator as o

#######################################################################
#
# The base class representation of a Graph with all the interface
# methods
#
#######################################################################
class Graph(abc.ABC):

    def __init__(self, numVertices, directed=False):
        self.numVertices = numVertices
        self.directed = directed

    @abc.abstractmethod
    def add_edge(self, v1, v2, weight):
        pass

    @abc.abstractmethod
    def get_adjacent_vertices(self, v):
        pass

    @abc.abstractmethod
    def get_indegree(self, v):
        pass

    @abc.abstractmethod
    def get_edge_weight(self, v1, v2):
        pass

    @abc.abstractmethod
    def display(self):
        pass


#######################################################################
#
# Represents a graph as an adjacency matrix. A cell in the matrix has
# a value when there exists an edge between the vertex represented by
# the row and column numbers.
# Weighted graphs can hold values > 1 in the matrix cells
# A value of 0 in the cell indicates that there is no edge
#
#######################################################################
class AdjacencyMatrixGraph(Graph):

    def __init__(self, numVertices, directed=False):
        super(AdjacencyMatrixGraph, self).__init__(numVertices, directed)

        self.matrix = np.zeros((numVertices, numVertices))


    def add_edge(self, v1, v2, weight=1):
        if v1 >= self.numVertices or v2 >= self.numVertices or v1 < 0 or v2 < 0:
            raise ValueError("Vertices %d and %d are out of bounds" % (v1, v2))

        if weight < 1:
            raise ValueError("An edge cannot have weight < 1")

        self.matrix[v1][v2] = weight
        if self.directed == False:
            self.matrix[v2][v1] = weight

    def get_adjacent_vertices(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)

        adjacent_vertices = []
        for i in range(self.numVertices):
            if self.matrix[v][i] > 0:
                adjacent_vertices.append(i)

        return adjacent_vertices

    def get_indegree(self, v):
        if v < 0 or v >= self.numVertices:
            raise ValueError("Cannot access vertex %d" % v)

        indegree = 0
        for i in range(self.numVertices):
            if self.matrix[i][v] > 0:
                indegree = indegree + 1

        return indegree

    def get_edge_weight(self, v1, v2):
        return self.matrix[v1][v2]

    def display(self):
        for i in range(self.numVertices):
            for v in self.get_adjacent_vertices(i):
                print(i, "-->", v)


def UCS(graph, start, goal):
    q = []
    q.append([0,start])
    traverseCost = 0
    visited = []
    while ( len(q) > 0 ):
        q.sort(key = o.itemgetter(0))
        cost, node = q.pop(0)
        visited.append(node)
        print("Visited Node:", node)
        adjacent_vertices = g.get_adjacent_vertices(node)
        traverseCost += cost
        if ( node == goal ):
            print("Path is :", visited)
            return traverseCost
        for x in adjacent_vertices:
            if x not in visited:
                q.append([g.get_edge_weight(node,x),x])   
        
if __name__ == "__main__":
    g = AdjacencyMatrixGraph(10,True)
    g.add_edge(0, 1, 1)
    g.add_edge(0, 2, 1)
    g.add_edge(1, 3, 3)
    g.add_edge(2, 5, 2)
    g.add_edge(3, 6, 4)
    g.add_edge(3, 5, 2)
    g.add_edge(4, 6, 1)
    g.add_edge(4, 7, 5)
    g.add_edge(5, 4, 4)
    g.add_edge(6, 7, 1)
    g.add_edge(5, 0, 3)
    g.add_edge(5, 8, 1)
    g.add_edge(8, 4, 1)
    g.add_edge(8, 9, 3)
    g.add_edge(9, 7, 1)
    g.add_edge(7, 0, 30)

    for i in range(9):
         for j in g.get_adjacent_vertices(i):
                print("Edge weight: ", i, " ", j, " weight: ", g.get_edge_weight(i, j))

    g.display()
    cost = UCS(g,0,8)
    print("Cost is : " + str(cost))

In [ ]:
import operator
# This class represent a graph
class Graph:
    # Initialize the class
    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()
                
    # Create an undirected graph by adding symmetric edges
    def make_undirected(self):
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.graph_dict.setdefault(b, {})[a] = dist
                    
    # Add a link from A and B of given distance, and also add the inverse link if the graph is undirected
    def connect(self, A, B, distance=1):
        self.graph_dict.setdefault(A, {})[B] = distance
        if not self.directed:
            self.graph_dict.setdefault(B, {})[A] = distance
                
    # Get neighbors or a neighbor
    def get(self, a, b=None):
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)     
    # Return a list of nodes in the graph
    def nodes(self):
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)
        
# This class represent a node
class Node:    
    # Initialize the class
    def __init__(self, name, parent):
        self.name = name
        self.parent = parent
        self.g = 0 # Distance to start node
        self.h = 0 # Distance to goal node
        self.f = 0 # Total cost
        self.position = 0
    def __lt__(self,other):
        return self.f < other.f
    def __eq__(self, other):
        return self.name == other.name
    def __repr__(self):
        return ('({0},{1})'.format(self.position, self.f))
        
# A* search
def A_Star(graph, heuristics, start, end):
    open1 = []
    closed = []
    starter = Node(start,"Yateem")  
    starter.g = 0
    starter.f = 0
    starter.h = heuristics[starter.name]
    open1.append(starter)
    
    while ( len(open1) > 0 ):
        open1 = sorted(open1, key=lambda x: x.f)
        current_node = open1.pop(0)
        closed.append(current_node)
        if ( current_node.name == end):
            return closed[::-1]
        neighbors = graph.get(current_node.name)
        for key, value in neighbors.items():
            neighbor = Node(key,current_node.name)
            if(neighbor in closed):
                continue
            neighbor.h = heuristics[key]
            neighbor.g = value + current_node.g
            neighbor.f = neighbor.g + neighbor.h
            if(In_Open(open1, neighbor) == True):
                open1.append(neighbor)
    return False   
def In_Open(open, neighbor):
    for node in open:
        if (neighbor == node and neighbor.f >= node.f):
            return False
    return True
    
def reversePrint(path,start,current):   
    if ( current != start):
        for x in path:
            if ( x.name == current):
                reversePrint(path,start,x.parent)
                break
        print(current)
    else:
        print(current)      
# The main entry point for this module
def main():
    # Create a graph
    graph = Graph()
        
    # Create graph connections (Actual distance)
    # Add Remaining Links From Example Given in Sides (Romania Map)
    graph.connect('Fagaras', 'Bucharest', 211)
    graph.connect('Pitesti', 'Bucharest', 101)
    graph.connect('Giurgiu', 'Bucharest', 90)
    graph.connect('Oradea', 'Zerind', 71)   
    graph.connect('Oradea', 'Sibiu', 151)
    graph.connect('Zerind', 'Arad', 75)
    graph.connect('Arad', 'Timisoara', 118)
    graph.connect('Arad', 'Sibiu', 140)
    graph.connect('Timisoara', 'Lugoj', 111)
    graph.connect('Lugoj', 'Mehadia', 70)
    graph.connect('Mehadia', 'Dobreta', 75)
    graph.connect('Dobreta', 'Craoiva', 120)
    graph.connect('Craoiva', 'Pitesti', 138)
    graph.connect('Sibiu', 'Fagaras', 99)
    graph.connect('Sibiu', 'Rimnicu Vilcea', 80)
    graph.connect('Rimnicu Vilcea', 'Pitesti', 97)
    graph.connect('Rimnicu Vilcea', 'Craoiva', 97)
    # Make graph undirected, create symmetric connections
    graph.make_undirected()
        
    # Create heuristics (straight-line distance, air-travel distance) for Destination Bucharest
    heuristics = {}
    heuristics['Arad'] = 366
    heuristics['Bucharest'] = 0
    heuristics['Zerind'] = 374
    heuristics['Timisoara'] = 329
    heuristics['Oradea'] = 380
    heuristics['Lugoj'] = 244
    heuristics['Mehadia'] = 241
    heuristics['Dobreta'] = 242
    heuristics['Craoiva'] = 160
    heuristics['Sibiu'] = 253
    heuristics['Rimnicu Vilcea'] = 193
    heuristics['Pitesti'] = 10
    heuristics['Fagaras'] = 176
    # Print Graph Nodes
    print(graph.nodes())
    print('\n')
        
    # Run search algorithm
    start = 'Arad'
    end = 'Bucharest'
    path = A_Star(graph, heuristics, start, end)
    if ( path == False ):
        print("Path not found")
    else:   
        name = end
        print("Nodes traversed in reverse are: ")
        while ( name != start):
            for x in path:
                if ( name == x.name ):
                    name = x.parent
                    print(name)
                    break

        
        
# Tell python to run main method
if __name__ == "__main__": main()



In [23]:
from math import *
import numpy as np

class board:
    def __init__(self, n):
        self.n = n
        self.array = np.zeros((n, n), dtype=np.int8)
        self.position = []
        
        count = 0
        
        while count < n:
            x = np.random.randint(0, n)
            y = np.random.randint(0, n)
            
            if self.array[y, x] > 0:
                continue
            self.array[y, x] = count+1
            self.position.append([y, x])
            
            count += 1
    
    def evaluate(self):
        count = 0
        for i in range(self.n):
            for j in range(self.n):
                cur = self.array[i, j]
                if(cur > 0):
                    count += self.evaluate_cur(i, j, cur)
        
        if (count%2 != 0):
            print(f"Something is very wrong")
        
        return floor(count/2)
    
    def evaluate_cur(self, ii, jj, cur):
        count = 0
        for i in range(self.n):
            for j in range(self.n):
                cur2 = self.array[i, j]
                if cur2 > 0 and cur2 != cur:
                    dist_1 = abs(ii-i)
                    dist_2 = abs(jj-j)
                    if ii == i or jj == j or dist_1 == dist_2:
                        count += 1
        return count
    
    def print(self):
        #print(self.array)
        print("-------------------------")
        for i in range(self.n):
            for j in range(self.n):
                if(self.array[i, j] > 0):
                    print(f" {self.array[i, j]} ", end="")
                else:
                    print(f" . ", end="")
            
            print(f"")
        print("-------------------------")
            
    def hill_climb(self):
        #Stochastic Hill Climbing
        
        iterations = 100
        
        while iterations > 0:
            #Current violations
            cur = self.evaluate()
            
            #Queen to Move
            queen = np.random.randint(0, self.n)

            #Queen Possible moves
            moves = []
            y_pos = self.position[queen][0]
            x_pos = self.position[queen][1]
            
            for i in range(self.n):
                if (i != y_pos and self.array[i, x_pos] == 0):
                    moves.append([i, x_pos])
                
            for j in range(self.n):
                if (j != x_pos and self.array[y_pos, j] == 0):
                    moves.append([y_pos, j])
                    
            #Picking a random move
            move = moves[np.random.randint(0, len(moves))]
            
            npback = self.array
            
            self.array = np.copy(npback)
            self.array[y_pos, x_pos] = 0
            self.array[move[0], move[1]] = queen+1
            
            #New violations
            cur_new = self.evaluate()
            
            if(cur_new < cur):
                self.position[queen] = move
            else:
                self.array = npback
            
            iterations -= 1
            
            #self.print()
        
        

if __name__ == "__main__":
    Board = board(8)
    Board.print()
    print(Board.evaluate())
    
    Board.hill_climb()
    Board.print()
    print(Board.evaluate())


-------------------------
 4  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  . 
 .  .  .  3  .  .  .  . 
 .  .  .  .  2  .  .  . 
 5  .  .  .  .  .  .  . 
 .  .  .  1  .  .  .  . 
 .  .  8  .  .  .  .  . 
 .  .  .  .  .  6  .  . 
-------------------------
8
-------------------------
 4  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  . 
 .  .  .  3  .  .  .  . 
 .  .  .  .  2  .  .  . 
 5  .  .  .  .  .  .  . 
 .  .  .  1  .  .  .  . 
 .  .  8  .  .  .  .  . 
 .  .  .  .  .  6  .  . 
-------------------------
-------------------------
 4  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  . 
 .  .  .  3  .  .  .  . 
 .  .  .  .  2  .  .  . 
 5  .  .  .  .  .  .  . 
 .  .  .  1  .  .  .  . 
 .  .  8  .  .  .  .  . 
 .  .  .  .  .  6  .  . 
-------------------------
-------------------------
 4  .  .  .  .  .  .  . 
 7  .  .  .  .  .  .  . 
 .  .  .  3  .  .  .  . 
 .  .  .  .  2  .  .  . 
 5  .  .  .  .  .  .  . 
 .  .  .  1  .  .  .  . 
 .  .  8  .  .  .  .  . 
 .  .  .  .  .  6  .  . 
----------------